## 1. Place Human, Furnitures in the Mujoco Simulator

In [ ]:

import sys,mujoco,time,os,json
import numpy as np
import matplotlib.pyplot as plt
sys.path.append('../package/helper/')
sys.path.append('../package/mujoco_usage/')
sys.path.append('../package/gpt_usage/')
sys.path.append('../package/detection_module/')
from mujoco_parser import *
from utility import *
from transformation import *
from gpt_helper import *
from owlv2 import *
np.set_printoptions(precision=2,suppress=True,linewidth=100)
plt.rc('xtick',labelsize=6); plt.rc('ytick',labelsize=6)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("Ready.")

# Joint configuration
joint_names = ['shoulder_pan_joint','shoulder_lift_joint','elbow_joint',
               'wrist_1_joint','wrist_2_joint','wrist_3_joint']
print ("Ready.")

In [ ]:
xml_path = '../asset/makeup_frida/scene_table.xml'
env = MuJoCoParserClass(name='Tabletop',rel_xml_path=xml_path,verbose=True)
print ("Done.")
# Solve IK to get the initial position 


env.reset()
env.set_p_body(body_name='ur_base',p=np.array([0,0,0.5])) # move UR
q_init,ik_err_stack,ik_info = solve_ik(
    env = env,
    joint_names_for_ik = joint_names,
    body_name_trgt     = 'ur_camera_center',
    q_init       = np.deg2rad([0,0,0,0,0,0]), # ik from zero pose
    p_trgt       = np.array([0.41,0.0,1.2]),
    R_trgt       = rpy2r(np.deg2rad([-135.22,-0.,-90])),
    max_ik_tick  = 5000,
    ik_err_th    = 1e-4,
    ik_stepsize  = 0.1,
    ik_eps       = 1e-2,
    ik_th        = np.radians(1.0),
    verbose      = False,
    reset_env    = False,
    render       = False,
    render_every = 1,
)
print ("Done.")

In [ ]:
# np.random.seed(seed=0)

env.reset()
env.init_viewer(
    transparent = False,
    azimuth     = 105,
    distance    = 3.12,
    elevation   = -29,
    lookat      = [0.39, 0.25, 0.43],
)
env.set_p_body(body_name='ur_base',p=np.array([0,0,0.5])) # move UR
env.set_p_body(body_name='object_table',p=np.array([1.0,0,0])) # move table
# obj_names = env.get_body_names(prefix='obj_') # object names
obj_names = ['obj_head']
n_obj = len(obj_names)
obj_xyzs = sample_xyzs(
    n_obj,
    x_range   = [0.75,1.25],
    y_range   = [-0.4,+0.4],
    z_range   = [0.51,0.51],
    min_dist  = 0.1,
    xy_margin = 0.0
)
obj_xyzs = np.array([[1, 0, 0.51]])
R = rpy2r(np.radians([0,0,270]))
print ("Object list:")
for obj_idx in range(n_obj):
    print (" [%d] obj_name:[%s]"%(obj_idx,obj_names[obj_idx]))
    env.set_p_base_body(body_name=obj_names[obj_idx],p=obj_xyzs[obj_idx,:])
    env.set_R_base_body(body_name=obj_names[obj_idx],R=R)
# Move
qpos = np.radians([0,-90,60,75,90,0])
idxs_step = env.get_idxs_step(joint_names=joint_names)
env.set_qpos_joints(joint_names=joint_names,qpos=q_init)
# Loop
env_state = env.get_state()
while env.is_viewer_alive():
    # Step
    env.step(ctrl=q_init,ctrl_idxs=idxs_step)
    # Grab RGB-D 
    if env.loop_every(tick_every=10):
        p_cam,R_cam = env.get_pR_body(body_name='ur_camera_center')
        p_ego  = p_cam
        p_trgt = p_cam + R_cam[:,2] # z-axis forward
        ego_rgb_img,ego_depth_img,pcd,xyz_img,xyz_img_world = env.get_egocentric_rgbd_pcd(
            p_ego            = p_ego,
            p_trgt           = p_trgt,
            rsz_rate_for_pcd = 1/20, # 1/4
            rsz_rate_for_img = 1/4,
            fovy             = 45, # env.model.cam_fovy[0]
            restore_view     = True,
        )
    # Render
        p_ego  = p_cam
        p_trgt = p_cam + R_cam[:,2] # z-axis forward
        ego_rgb_img,ego_depth_img,pcd,xyz_img,xyz_img_world = env.get_egocentric_rgbd_pcd(
            p_ego            = p_ego,
            p_trgt           = p_trgt,
            rsz_rate_for_pcd = 1/20, # 1/4
            rsz_rate_for_img = 1/4,
            fovy             = 45, # env.model.cam_fovy[0]
            restore_view     = True,
        )

    if env.loop_every(tick_every=500): # every 1 second
        # Grab current view
        render_img = env.grab_image(rsz_rate=1/4)
        # Plot
        plt.figure(figsize=(8,3))
        plt.subplot(121)
        plt.imshow(render_img)
        plt.subplot(122)
        plt.imshow(ego_rgb_img)
        plt.suptitle("Time:[%.2f]sec"%(env.get_sim_time()),fontsize=10)
        # plt.tight_layout()
        plt.subplots_adjust(wspace=0.1, hspace=0.1, top=0.99)
        plt.show()
        
env.close_viewer()
print ("Done.")
env.close_viewer()
print ("Done.")